In [3]:
import pandas as pd
import os
from tqdm import tqdm
from data_wrapper import FootballPredictorDataWrapper, add_h2h_goals, add_h2h_results, add_h2h_stats

In [2]:
#this method takes every csv from every season and every league that will be used for modelling and runs data_wrapper.py on it
def prepare_data():
    for league_folder in tqdm(os.listdir('./modelling_data/raw_data')):
        for csv_file in tqdm(os.listdir(f'./modelling_data/raw_data/{league_folder}')):
            try:
                df = pd.read_csv(f'./modelling_data/raw_data/{league_folder}/{csv_file}')
                wrapper = FootballPredictorDataWrapper(df)
                df_prepared = wrapper.run()
                df_prepared.to_csv(f'./modelling_data/prepared_data/{league_folder}/{csv_file.split(".")[0]}-prepared.csv')
            except Exception as e:
                print(f'There was a problem with preparing data from {csv_file} -> {e}')

In [3]:
prepare_data()

  0%|          | 0/20 [00:00<?, ?it/s]

c:\Users\Łukasz\magisterka infa\football_games_predictor\data_wrapper.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_GMT'] = pd.to_datetime(df['date_GMT'])
c:\Users\Łukasz\magisterka infa\football_games_predictor\data_wrapper.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_GMT'] = pd.to_datetime(df['date_GMT'])
c:\Users\Łukasz\magisterka infa\football_games_predictor\data_wrapper.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date_GMT'] = pd.to_datetime(df['date_GMT'])
c:\Users\Łukasz\magisterka infa\football_games_predictor\data_wrapp

In [6]:
#this method takes every prepared csv from every season and every league that will be used for modelling, joins data from different seasons and adds h2h features
def join_data_and_add_h2h_features():
    for league_folder in tqdm(os.listdir('./modelling_data/prepared_data')):
        league_data_all_seasons = pd.DataFrame()
        for csv_file in tqdm(os.listdir(f'./modelling_data/prepared_data/{league_folder}')):
            data_from_specific_season = pd.read_csv(f'./modelling_data/prepared_data/{league_folder}/{csv_file}')
            league_data_all_seasons = pd.concat([league_data_all_seasons, data_from_specific_season], axis=0, ignore_index=True)
            
        league_data_all_seasons = add_h2h_results(league_data_all_seasons)
        league_data_all_seasons = add_h2h_goals(league_data_all_seasons)
        league_data_all_seasons = add_h2h_stats(league_data_all_seasons)
        league_data_all_seasons.drop(['team_home_x', 'team_away_x', 'team_home_y', 'team_away_y'], axis=1, inplace=True)
        
        league_data_all_seasons.to_csv(f'./modelling_data/concatenated_data/{league_folder}_data_prepared_all_seasons.csv', index=False)    

In [7]:
join_data_and_add_h2h_features()

100%|██████████| 20/20 [00:06<00:00,  2.94it/s]


In [8]:
def final_data_cleaning(data):
    try:
        data.drop('Unnamed: 0', axis=1, inplace=True)
    except KeyError:
        print('There is no old index column in dataframe.')
    
    """
    filling empties in stadium attendance
    """
    
    #step 1 - filling empties with mean value for home team and season
    data['attendance'] = data['attendance'].fillna(data.groupby(['home_team_name', 'season'])['attendance'].transform('mean'))
    
    #step 2 (optional) - filling empties with mean value for whole league and stadium
    data['attendance'] = data['attendance'].fillna(data.groupby(['league', 'season'])['attendance'].transform('mean'))

    #step 3 (optional) - filling empties with mean value for whole league across all seasons
    data['attendance'] = data['attendance'].fillna(data.groupby(['league'])['attendance'].transform('mean'))
    
    """
    renaming columns
    """

    data.rename(columns={
        'Game Week': 'game_week',
        'team_a_xg': 'home_team_xg',
        'team_b_xg': 'away_team_xg'
    }, inplace=True)
    
    data.columns = [col.lower() for col in data.columns]    

    """
    removing unnecessary features
    """
    
    features_to_drop = [
        'referee',
        'home_ppg',
        'away_ppg',
        'stadium_name'
    ]
    
    data.drop(features_to_drop, axis=1, inplace=True)
    
    return data

In [ ]:
for csv_file in os.listdir('./modelling_data/concatenated_data/')

In [15]:
root

'./modelling_data/concatenated_data/cleaned_data'

In [18]:
os.path.join(root, name)

'./modelling_data/concatenated_data/cleaned_data\\ukraine_data_prepared_all_seasons.csv'

In [19]:
os.path.join('./modelling_data/concatenated_data', 'cleaned_data', 'data.csv')

'./modelling_data/concatenated_data\\cleaned_data\\data.csv'

In [25]:
for root, _, files in os.walk('./modelling_data/concatenated_data/'):
    for name in tqdm(files):
        csv_file_path = os.path.join(root, name)
        data = pd.read_csv(csv_file_path)
        data_clean = final_data_cleaning(data)
        data_clean.to_csv(os.path.join(root, 'cleaned_data', f'{name.split(".")[0]}_cleaned.csv'), index=False)

  0%|          | 0/20 [00:00<?, ?it/s]

There is no old index column in dataframe.


KeyError: "['referee', 'home_ppg', 'away_ppg', 'stadium_name'] not found in axis"

In [31]:
all_data = pd.DataFrame()

for csv_file in os.listdir('./modelling_data/concatenated_data/cleaned_data'):
    data_from_specific_league = pd.read_csv(f'./modelling_data/concatenated_data/cleaned_data/{csv_file}')
    all_data = pd.concat([all_data, data_from_specific_league], axis=0)
    
all_data.to_csv('./modelling_data/concatenated_data/cleaned_data/all_games_1819_2324.csv', index=False)

In [32]:
data = pd.read_csv('./modelling_data/concatenated_data/cleaned_data/all_games_1819_2324.csv')

In [33]:
data.shape

(37746, 162)

In [35]:
data['season'].value_counts()

season
21/22    6433
20/21    6338
22/23    6254
23/24    6235
18/19    6095
19/20    5688
17/18     463
24/25     240
Name: count, dtype: int64